In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, avg, sum as spark_sum, max as spark_max, min as spark_min,
    current_timestamp, lit, when, round as spark_round, to_date, weekofyear, year,
    month, dayofmonth, row_number, percentile_approx, collect_list, struct, countDistinct
)
from pyspark.sql.types import IntegerType, StringType, TimestampType, DateType, FloatType, DecimalType
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F

In [0]:
catalog_name = "atendimento_catalog"
silver_db_name = "silver"
gold_db_name = "gold"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {gold_db_name}")

## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.

In [0]:
def safe_table_exists(full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

def safe_col(df, name):
    return col(name) if name in df.columns else lit(None).cast(StringType())

## Processamento: fact_chamados (Tabela Fato Principal)

Tabela fato linha-a-linha com todos os chamados enriquecidos com dimensões.
Cada registro representa um único chamado, sem agregações.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias para enriquecimento.

In [0]:
src_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_clientes = f"{catalog_name}.{silver_db_name}.dm_clientes"
src_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_motivos = f"{catalog_name}.{silver_db_name}.dm_motivos"
src_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"
src_table_custos = f"{catalog_name}.{silver_db_name}.ft_custos"
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_fact = f"{catalog_name}.{gold_db_name}.fact_chamados"

if not safe_table_exists(src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")

df_chamados_src = spark.table(src_table_chamados)
df_clientes_src = spark.table(src_table_clientes) if safe_table_exists(src_table_clientes) else None
df_canais_src = spark.table(src_table_canais) if safe_table_exists(src_table_canais) else None
df_motivos_src = spark.table(src_table_motivos) if safe_table_exists(src_table_motivos) else None
df_atendentes_src = spark.table(src_table_atendentes) if safe_table_exists(src_table_atendentes) else None
df_custos_src = spark.table(src_table_custos) if safe_table_exists(src_table_custos) else None
df_satisfacao_src = spark.table(src_table_satisfacao) if safe_table_exists(src_table_satisfacao) else None

total_before_chamados = df_chamados_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros ft_chamados: {total_before_chamados:,}")

### Construção da Tabela Fato Linha-a-Linha

Join com dimensões mantendo granularidade original (um registro por chamado).

In [0]:
df_fact = df_chamados_src

if df_clientes_src is not None:
    df_fact = df_fact.alias("ch").join(
        df_clientes_src.alias("cl"),
        col("ch.id_cliente") == col("cl.id_cliente"),
        "left"
    ).select(
        col("ch.*"),
        col("cl.regiao")
    )
else:
    df_fact = df_fact.withColumn("regiao", lit(None).cast(StringType()))

if df_canais_src is not None:
    df_fact = df_fact.alias("f").join(
        df_canais_src.alias("ca"),
        col("f.id_canal") == col("ca.id_canal"),
        "left"
    ).select("f.*", col("ca.id_canal").alias("id_canal_dim"))
else:
    df_fact = df_fact.withColumn("id_canal_dim", lit(None).cast(IntegerType()))

if df_motivos_src is not None:
    df_fact = df_fact.alias("f").join(
        df_motivos_src.alias("m"),
        col("f.id_motivo") == col("m.id_motivo"),
        "left"
    ).select("f.*")
else:
    pass

if df_atendentes_src is not None:
    df_fact = df_fact.alias("f").join(
        df_atendentes_src.alias("a"),
        col("f.id_atendente") == col("a.id_atendente"),
        "left"
    ).select("f.*")
else:
    pass

if df_custos_src is not None:
    df_fact = df_fact.alias("f").join(
        df_custos_src.alias("cu"),
        col("f.id_chamado") == col("cu.id_chamado"),
        "left"
    ).select(
        col("f.*"),
        col("cu.custo")
    )
else:
    df_fact = df_fact.withColumn("custo", lit(None))

if df_satisfacao_src is not None:
    df_fact = df_fact.alias("f").join(
        df_satisfacao_src.alias("s"),
        col("f.id_chamado") == col("s.id_chamado"),
        "left"
    ).select(
        col("f.*"),
        col("s.nota_atendimento")
    )
else:
    df_fact = df_fact.withColumn("nota_atendimento", lit(None).cast(IntegerType()))

df_fact = df_fact.withColumn("data", to_date(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("semana", weekofyear(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("ano", year(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("processed_timestamp", current_timestamp())

df_fact_typed = df_fact \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("id_atendente", col("id_atendente").cast(IntegerType())) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("id_canal", col("id_canal").cast(IntegerType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("resolvido", col("resolvido").cast(StringType())) \
    .withColumn("tempo_espera_minutos", col("tempo_espera_minutos").cast(FloatType())) \
    .withColumn("tempo_atendimento_minutos", col("tempo_atendimento_minutos").cast(FloatType())) \
    .withColumn("custo", col("custo").cast(DecimalType(18, 8))) \
    .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_fact_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_fact)

fact_chamados = spark.table(tgt_table_fact)
final_count_fact = fact_chamados.count()

print(f"Salvo em: {tgt_table_fact}")
print(f"Registros na tabela fato: {final_count_fact:,}")

### Agregações Analíticas Derivadas da Tabela Fato

Cálculo de métricas de funil baseado na tabela fato linha-a-linha.

In [0]:
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"

df_funnel = fact_chamados.withColumn(
    "n_autosservico",
    when(
        (col("canal").isin(["Ura", "Chatbot", "Web"])) & 
        (col("id_atendente").isNull()) &
        (col("hora_finalizacao_atendimento").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
).withColumn(
    "n_nivel1",
    when(col("id_atendente").isNotNull(), 1).otherwise(0).cast(IntegerType())
).withColumn(
    "n_resolvido",
    when(col("resolvido") == "Sim", 1).otherwise(0).cast(IntegerType())
)

df_funnel_agg = df_funnel.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_resolvido").alias("n_resolvido"),
    count("id_chamado").alias("volume_total_chamados"),
    spark_round(avg("tempo_espera_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_espera_medio"),
    spark_round(avg("tempo_atendimento_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_atendimento_medio"),
    spark_round(spark_sum("custo"), 2).cast(DecimalType(10, 2)).alias("custo_total")
)

df_funnel_agg = df_funnel_agg.withColumn("processed_timestamp", current_timestamp())

df_funnel_typed = df_funnel_agg \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_resolvido", col("n_resolvido").cast(IntegerType())) \
    .withColumn("volume_total_chamados", col("volume_total_chamados").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", col("tempo_espera_medio").cast(DecimalType(10, 2))) \
    .withColumn("tempo_atendimento_medio", col("tempo_atendimento_medio").cast(DecimalType(10, 2))) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Registros agregados: {final_count_funnel:,}")

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: anomalous_escalations

Identificação de chamados com escalações anômalas.
Detecção de chamados que escalaram sem passar por autosserviço.

### Filtro de Anomalias

Seleção de chamados que não passaram por autosserviço mas foram escalados.

In [0]:
tgt_table_anomalous = f"{catalog_name}.{gold_db_name}.anomalous_escalations"

df_anomalous = fact_chamados.filter(
    (col("id_atendente").isNotNull())
).select(
    col("id_chamado"),
    col("id_cliente"),
    col("data"),
    col("canal"),
    col("id_motivo"),
    col("motivo"),
    lit("Escalação detectada").alias("tipo_anomalia")
).withColumn("processed_timestamp", current_timestamp())

df_anomalous_typed = df_anomalous \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("tipo_anomalia", col("tipo_anomalia").cast(StringType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de anomalias detectadas: {df_anomalous_typed.count():,}")

df_anomalous_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_anomalous)

gold_table_anomalous = spark.table(tgt_table_anomalous)
final_count_anomalous = gold_table_anomalous.count()

print(f"Salvo em: {tgt_table_anomalous}")
print(f"Contagem final: {final_count_anomalous:,}")

### Gravação na Camada Gold

Persistência da tabela de anomalias em formato Delta.

## Processamento: top_reasons_volume

Top motivos por volume de chamados.

### Agregação por Motivo

Contagem de chamados por motivo, derivando do fact_chamados.

In [0]:
tgt_table_top_volume = f"{catalog_name}.{gold_db_name}.top_reasons_volume"

# Agregação do fact_chamados por motivo
df_top_volume = fact_chamados.groupBy("id_motivo", "motivo").agg(
    count("id_chamado").alias("volume_chamados")
).orderBy(col("volume_chamados").desc())

df_top_volume = df_top_volume.withColumn("processed_timestamp", current_timestamp())

df_top_volume_typed = df_top_volume \
    .withColumn("id_motivo", col("id_motivo").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_volume_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por volume.

In [0]:
df_top_volume_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_volume)

gold_table_top_volume = spark.table(tgt_table_top_volume)
final_count_top_volume = gold_table_top_volume.count()

print(f"Salvo em: {tgt_table_top_volume}")
print(f"Contagem final: {final_count_top_volume:,}")

display(gold_table_top_volume.limit(10))

## Processamento: top_reasons_cost

Top motivos por custo de atendimento.

### Agregação por Motivo e Custo

Soma de custos por motivo, derivando do fact_chamados.

In [0]:
tgt_table_top_cost = f"{catalog_name}.{gold_db_name}.top_reasons_cost"

# Agregação do fact_chamados por motivo com custos
df_top_cost = fact_chamados.groupBy("id_motivo", "motivo").agg(
    spark_round(spark_sum("custo"), 2).alias("custo_total"),
    count("id_chamado").alias("volume_chamados")
).orderBy(col("custo_total").desc())

df_top_cost = df_top_cost.withColumn(
    "custo_medio",
    spark_round(
        when(
            col("volume_chamados") > 0,
            col("custo_total") / col("volume_chamados")
        ).otherwise(lit(None)),
        2
    )
)

df_top_cost = df_top_cost.withColumn("processed_timestamp", current_timestamp())

df_top_cost_typed = df_top_cost \
    .withColumn("id_motivo", col("id_motivo").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(10, 2))) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("custo_medio", col("custo_medio").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))


print(f"Total de motivos únicos: {df_top_cost_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por custo.

In [0]:
df_top_cost_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_cost)

gold_table_top_cost = spark.table(tgt_table_top_cost)
final_count_top_cost = gold_table_top_cost.count()

print(f"Salvo em: {tgt_table_top_cost}")
print(f"Contagem final: {final_count_top_cost:,}")

display(gold_table_top_cost.limit(10))

## Processamento: top_reasons_low_satisfaction

Top motivos com menor satisfação do cliente.

### Leitura e Join com Satisfação

Join do fact_chamados com tabela de satisfação para análise por motivo.

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_top_satisfaction = f"{catalog_name}.{gold_db_name}.top_reasons_low_satisfaction"

if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

# Join fact_chamados com satisfação
df_base_satisfaction = fact_chamados.alias("f") \
    .join(df_satisfacao_src.alias("s"), col("f.id_chamado") == col("s.id_chamado"), "inner") \
    .select(
        col("f.id_chamado"),
        col("f.id_motivo"),
        col("f.motivo"),
        col("s.nota_atendimento").alias("nota_satisfacao")
    )

print(f"Registros com pesquisa de satisfação: {df_base_satisfaction.count():,}")

### Agregação por Motivo

Cálculo de nota média por motivo, derivando do fact_chamados com satisfação.

In [0]:
df_top_satisfaction = df_base_satisfaction.groupBy("id_motivo", "motivo").agg(
    spark_round(avg("nota_satisfacao"), 2).alias("nota_media"),
    count("id_chamado").alias("volume_respostas"),
    spark_sum(
        when(col("nota_satisfacao") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao")
).orderBy(col("nota_media").asc())

df_top_satisfaction = df_top_satisfaction.withColumn(
    "perc_baixa_satisfacao",
    spark_round(
        when(
            col("volume_respostas") > 0,
            (col("count_baixa_satisfacao") / col("volume_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_top_satisfaction = df_top_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_top_satisfaction_typed = df_top_satisfaction \
    .withColumn("id_motivo", col("id_motivo").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("count_baixa_satisfacao", col("count_baixa_satisfacao").cast(IntegerType())) \
    .withColumn("perc_baixa_satisfacao", col("perc_baixa_satisfacao").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_satisfaction_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de motivos com baixa satisfação.

In [0]:
df_top_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_satisfaction)

gold_table_top_satisfaction = spark.table(tgt_table_top_satisfaction)
final_count_top_satisfaction = gold_table_top_satisfaction.count()

print(f"Salvo em: {tgt_table_top_satisfaction}")
print(f"Contagem final: {final_count_top_satisfaction:,}")

display(gold_table_top_satisfaction.limit(10))

## Processamento: satisfaction_trends

Tendências de satisfação ao longo do tempo por canal e região.

### Agregação Temporal

Cálculo de métricas de satisfação por data, canal e região derivando do fact_chamados.

In [0]:
tgt_table_satisfaction_trends = f"{catalog_name}.{gold_db_name}.satisfaction_trends"

# Join fact_chamados com satisfação para análise temporal
df_satisfaction_full = fact_chamados.alias("f") \
    .join(df_satisfacao_src.alias("s"), col("f.id_chamado") == col("s.id_chamado"), "inner") \
    .select(
        col("f.data"),
        col("f.semana"),
        col("f.ano"),
        col("f.canal"),
        col("f.regiao"),
        col("s.nota_atendimento").alias("nota_satisfacao")
    )

df_satisfaction = df_satisfaction_full.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_round(avg("nota_satisfacao"), 2).alias("nota_media"),
    count("nota_satisfacao").alias("total_respostas"),
    spark_sum(when(col("nota_satisfacao") >= 4, 1).otherwise(0)).alias("count_satisfeito"),
    spark_sum(when(col("nota_satisfacao") <= 3, 1).otherwise(0)).alias("count_insatisfeito")
)

df_satisfaction = df_satisfaction.withColumn(
    "perc_satisfeito",
    spark_round(
        when(
            col("total_respostas") > 0,
            (col("count_satisfeito") / col("total_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_satisfaction = df_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_typed = df_satisfaction \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("total_respostas", col("total_respostas").cast(IntegerType())) \
    .withColumn("count_satisfeito", col("count_satisfeito").cast(IntegerType())) \
    .withColumn("count_insatisfeito", col("count_insatisfeito").cast(IntegerType())) \
    .withColumn("perc_satisfeito", col("perc_satisfeito").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros: {df_satisfaction_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de tendências de satisfação.

In [0]:
df_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_trends)

gold_table_satisfaction_trends = spark.table(tgt_table_satisfaction_trends)
final_count_satisfaction_trends = gold_table_satisfaction_trends.count()

print(f"Salvo em: {tgt_table_satisfaction_trends}")
print(f"Contagem final: {final_count_satisfaction_trends:,}")

display(gold_table_satisfaction_trends.limit(10))

## Processamento: client_profiles

Perfis de clientes baseados em histórico de atendimento.

### Agregação por Cliente

Cálculo de métricas por cliente derivando do fact_chamados.

In [0]:
tgt_table_client_profiles = f"{catalog_name}.{gold_db_name}.client_profiles"

# Window para ranking de motivos por cliente
w_motivo = Window.partitionBy("id_cliente").orderBy(col("count_motivo").desc())

# Motivo principal por cliente (mais frequente)
df_motivo_por_cliente = fact_chamados.groupBy("id_cliente", "id_motivo", "motivo").agg(
    count("id_chamado").alias("count_motivo")
)

df_motivo_principal = df_motivo_por_cliente \
    .withColumn("rn", row_number().over(w_motivo)) \
    .filter(col("rn") == 1) \
    .select(col("id_cliente"), col("id_motivo"), col("motivo").alias("motivo_principal"))

# Agregação geral de clientes
df_client_profiles = fact_chamados.groupBy("id_cliente", "regiao").agg(
    count("id_chamado").alias("total_chamados"),
    spark_sum(when(col("resolvido") == "Sim", 1).otherwise(0)).alias("total_resolvidos"),
    spark_round(avg("tempo_espera_minutos"), 2).alias("tempo_espera_medio"),
    spark_round(spark_sum("custo"), 2).alias("custo_total_cliente")
)

df_client_profiles = df_client_profiles.join(df_motivo_principal, "id_cliente", "left")

df_client_profiles = df_client_profiles.withColumn(
    "taxa_resolucao",
    spark_round(
        when(
            col("total_chamados") > 0,
            (col("total_resolvidos") / col("total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_client_profiles = df_client_profiles.withColumn("processed_timestamp", current_timestamp())

df_client_profiles_typed = df_client_profiles \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("total_chamados", col("total_chamados").cast(IntegerType())) \
    .withColumn("total_resolvidos", col("total_resolvidos").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", col("tempo_espera_medio").cast(DecimalType(10, 2))) \
    .withColumn("custo_total_cliente", col("custo_total_cliente").cast(DecimalType(10, 2))) \
    .withColumn("id_motivo", col("id_motivo").cast(StringType())) \
    .withColumn("motivo_principal", col("motivo_principal").cast(StringType())) \
    .withColumn("taxa_resolucao", col("taxa_resolucao").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de clientes únicos: {df_client_profiles_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de perfis de clientes.

In [0]:
df_client_profiles_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_client_profiles)

gold_table_client_profiles = spark.table(tgt_table_client_profiles)
final_count_client_profiles = gold_table_client_profiles.count()

print(f"Salvo em: {tgt_table_client_profiles}")
print(f"Contagem final: {final_count_client_profiles:,}")

display(gold_table_client_profiles.limit(10))

## Processamento: satisfaction_correlations

relação da satisfação com outros dominios

### Select da tabela fato geral

Remove as informações desnecessarias para a analise da satisfação

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_satisfaction_correlations = f"{catalog_name}.{gold_db_name}.ft_correlacao_satisfacao"


if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

# Join fact_chamados com satisfação
# left porque tem chamadas sem avaliação
df_satisfaction_correlations = (
    fact_chamados.alias("f")
        .join(
            df_satisfacao_src.alias("s"),
            on=col("f.id_chamado") == col("s.id_chamado"),
            how="left"
        )
        .select(
            col("f.id_chamado"),
            col("s.id_pesquisa"),
            col("f.resolvido"),
            col("f.tempo_espera_minutos"),
            col("f.tempo_atendimento_minutos"),
            col("s.nota_atendimento").alias("nota_satisfacao"),
            col("f.data")
        )
        .withColumn("ano", year("data"))
        .withColumn("mes", month("data"))
        .withColumn("dia", dayofmonth("data"))
        .withColumn("semana", weekofyear("data"))
)

  

### Gravação na Camada Gold

Salvar ft_correlacao_satisfacao

In [0]:
df_satisfaction_correlations.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_correlations)

gold_table_satisfaction_correlations = spark.table(tgt_table_satisfaction_correlations)
final_count_satisfaction_correlations = gold_table_satisfaction_correlations.count()

print(f"Salvo em: {tgt_table_satisfaction_correlations}")
print(f"Contagem final: {final_count_satisfaction_correlations:,}")

display(gold_table_satisfaction_correlations.limit(10))

## Processamento: channel_satisfaction

relação da satisfação com tempo e canal

### Select da tabela fato geral

Remove as informações desnecessarias para a analise da satisfação

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_channel_satisfaction = f"{catalog_name}.{gold_db_name}.ft_canal_satisfacao"


if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

# Join fact_chamados com satisfação
df_channel_satisfaction = (
    fact_chamados.alias("f")
        .join(
            df_satisfacao_src.alias("s"),
            on=col("f.id_chamado") == col("s.id_chamado"),
            how="inner"
        )
        .select(
            col("s.id_pesquisa"),
            col("f.id_chamado"),
            col("f.id_canal"),
            col("f.canal"),
            col("s.nota_atendimento").alias("nota_satisfacao"),
            col("f.data")
        )
        .withColumn("ano", year("data"))
        .withColumn("mes", month("data"))
        .withColumn("dia", dayofmonth("data"))
        .withColumn("semana", weekofyear("data"))
)
  

### Gravação na Camada Gold

Salvar ft_canal_satisfacao

In [0]:
df_channel_satisfaction.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_channel_satisfaction)

gold_table_channel_satisfaction = spark.table(tgt_table_channel_satisfaction)
final_count_channel_satisfaction = gold_table_channel_satisfaction.count()

print(f"Salvo em: {tgt_table_channel_satisfaction}")
print(f"Contagem final: {final_count_channel_satisfaction:,}")

display(gold_table_channel_satisfaction.limit(10))

## Processamento: reason_satisfaction

relação da satisfação com tempo e canal

### Select da tabela fato geral

Remove as informações desnecessarias para a analise da satisfação

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_reason_satisfaction = f"{catalog_name}.{gold_db_name}.ft_canal_satisfacao"


if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

# Join fact_chamados com satisfação
df_reason_satisfaction = (
    fact_chamados.alias("f")
        .join(
            df_satisfacao_src.alias("s"),
            on=col("f.id_chamado") == col("s.id_chamado"),
            how="inner"
        )
        .select(
            col("s.id_pesquisa"),
            col("f.id_chamado"),
            col("f.id_motivo"),
            col("f.motivo"),
            col("s.nota_atendimento").alias("nota_satisfacao"),
            col("f.data")
        )
        .withColumn("ano", year("data"))
        .withColumn("mes", month("data"))
        .withColumn("dia", dayofmonth("data"))
        .withColumn("semana", weekofyear("data"))
)
  

### Gravação na Camada Gold

Salvar ft_canal_satisfacao

In [0]:
df_reason_satisfaction.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_reason_satisfaction)

gold_table_reason_satisfaction = spark.table(tgt_table_reason_satisfaction)
final_count_reason_satisfaction = gold_table_reason_satisfaction.count()

print(f"Salvo em: {tgt_table_reason_satisfaction}")
print(f"Contagem final: {final_count_reason_satisfaction:,}")

display(gold_table_reason_satisfaction.limit(10))

## Processamento: client_satisfaction

relação da satisfação com tempo e canal

### Select da tabela fato geral

Remove as informações desnecessarias para a analise da satisfação

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
src_table_clientes = f"{catalog_name}.{silver_db_name}.dm_clientes"
tgt_table_client_satisfaction = f"{catalog_name}.{gold_db_name}.ft_client_satisfacao"


if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

if not safe_table_exists(src_table_clientes):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_clientes}")

df_clientes_src = spark.table(src_table_clientes)

# Join fact_chamados com satisfação
df_client_satisfaction = (
    fact_chamados.alias("f")
        .join(
            df_satisfacao_src.alias("s"),
            on=col("f.id_chamado") == col("s.id_chamado"),
            how="inner"
        )
        .join(
            df_clientes_src.alias("c"),
            on=col("f.id_cliente") == col("c.id_cliente"),
            how="inner"
        )
        .select(
            col("s.id_pesquisa"),
            col("f.id_chamado"),
            col("f.id_cliente"),
            col("c.idade"),
            col("c.regiao"),
            col("s.nota_atendimento").alias("nota_satisfacao"),
            col("f.data")
        )
        .withColumn(
            "faixa_etaria",
            when(col("idade") < 25, "Até 24")
            .when((col("idade") >= 25) & (col("idade") <= 44), "25 a 44")
            .when((col("idade") >= 45) & (col("idade") <= 64), "45 a 64")
            .when(col("idade") >= 65, "65+")
            .otherwise("Não informado")
        )
        .withColumn("ano", year("data"))
        .withColumn("mes", month("data"))
        .withColumn("dia", dayofmonth("data"))
        .withColumn("semana", weekofyear("data"))
)
  

### Gravação na Camada Gold

Salvar ft_canal_satisfacao

In [0]:
df_client_satisfaction.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_client_satisfaction)

gold_table_client_satisfaction = spark.table(tgt_table_client_satisfaction)
final_count_client_satisfaction = gold_table_client_satisfaction.count()

print(f"Salvo em: {tgt_table_client_satisfaction}")
print(f"Contagem final: {final_count_client_satisfaction:,}")

display(gold_table_client_satisfaction.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO FINAL DE TRANSFORMAÇÃO - CAMADA GOLD")
print("="*80)
print(f"Tabelas Gold criadas: 11")
print(f"  - funnel_metrics: {final_count_funnel:,} registros")
print(f"  - anomalous_escalations: {final_count_anomalous:,} registros")
print(f"  - top_reasons_volume: {final_count_top_volume:,} registros")
print(f"  - top_reasons_cost: {final_count_top_cost:,} registros")
print(f"  - top_reasons_low_satisfaction: {final_count_top_satisfaction:,} registros")
print(f"  - satisfaction_trends: {final_count_satisfaction_trends:,} registros")
print(f"  - client_profiles: {final_count_client_profiles:,} registros")
print(f"  - satisfaction_correlations: {final_count_satisfaction_correlations:,} registros")
print(f"  - channel_satisfaction: {final_count_channel_satisfaction:,} registros")
print(f"  - reason_satisfaction: {final_count_reason_satisfaction:,} registros")
print(f"  - client_satisfaction: {final_count_client_satisfaction:,} registros")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: chamados_por_atendente

Quantidade de chamados por atendente

### Agregação por Atendente

Cálculo de métricas por atendente derivando do fact_chamados.

In [0]:
tgt_table_chamados_por_atendente = f"{catalog_name}.{gold_db_name}.chamados_por_atendente"

df_chamados_com_atendente = df_chamados_src.join(
    df_atendentes_src, 
    df_chamados_src["id_atendente"] == df_atendentes_src["id_atendente"],
    "inner"
).select(
    df_chamados_src["*"],
    df_atendentes_src["nome_atendente"],
    df_atendentes_src["nivel_atendimento"]
)


df_chamados_com_nota = df_chamados_src.join(
    df_satisfacao_src,
    "id_chamado",
    "left"
)

df_nota_media = (
    df_chamados_com_nota.groupBy("id_atendente")
    .agg(
        F.round(F.avg("nota_atendimento"), 2).alias("nota_media_atendente")
    )
)


df_chamados_por_atendente = (
    df_chamados_com_atendente.groupBy("id_atendente", "nome_atendente", "nivel_atendimento")  
    .agg(
        F.count("id_chamado").alias("total_chamados"),
        F.sum(F.when(F.col("resolvido") == "Sim", 1).otherwise(0)).alias("total_resolvidos"),
    )
)


df_chamados_por_atendente = (
    df_chamados_por_atendente
    .join(df_nota_media, "id_atendente", "left")
)

df_chamados_por_atendente = df_chamados_por_atendente.withColumn(
    "taxa_resolucao",
    F.round(F.col("total_resolvidos") * 100 / F.col("total_chamados"), 2)
).withColumn(
    "processed_timestamp", F.current_timestamp()
)


df_chamados_por_atendente_typed = df_chamados_por_atendente \
    .withColumn("id_atendente", F.col("id_atendente").cast(StringType())) \
    .withColumn("nome_atendente", F.col("nome_atendente").cast(StringType())) \
    .withColumn("nivel_atendimento", F.col("nivel_atendimento").cast(IntegerType())) \
    .withColumn("total_chamados", F.col("total_chamados").cast(IntegerType())) \
    .withColumn("total_resolvidos", F.col("total_resolvidos").cast(IntegerType())) \
    .withColumn("nota_media_atendente", F.col("nota_media_atendente").cast(DecimalType(10,2))) \
    .withColumn("taxa_resolucao", F.col("taxa_resolucao").cast(DecimalType(10,2))) \
    .withColumn("processed_timestamp", F.col("processed_timestamp").cast(TimestampType())) 


df_chamados_por_atendente_typed = df_chamados_por_atendente_typed.select(
    "id_atendente",
    "nome_atendente",
    "nivel_atendimento",
    "total_chamados",
    "total_resolvidos", 
    "taxa_resolucao",
    "nota_media_atendente",
    "processed_timestamp"
)


###Gravação na Camada Gold
Salvar chamados_por_atendente

In [0]:
df_chamados_por_atendente_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_chamados_por_atendente)

gold_table_chamados_por_atendente = spark.table(tgt_table_chamados_por_atendente)
final_count_chamados_por_atendente = gold_table_chamados_por_atendente.count()

print(f"Salvo em: {tgt_table_chamados_por_atendente}")
print(f"Contagem final: {final_count_chamados_por_atendente:,}")

display(gold_table_chamados_por_atendente.limit(10))

##Processamento: tempo_por_nivel
Medida de tempo medio de atendimento por nível 

In [0]:
tgt_table_tempo_por_nivel = f"{catalog_name}.{gold_db_name}.tempo_por_nivel"

taxa_por_nivel = (
    df_chamados_por_atendente_typed
    .groupBy("nivel_atendimento")
    .agg(
        F.round(F.avg("taxa_resolucao"), 2).alias("taxa_resolucao_media"),
        F.count("id_atendente").alias("total_atendentes"),
    )
    .withColumn("nivel_atendimento", F.col("nivel_atendimento").cast(IntegerType()))
    .orderBy("nivel_atendimento")
)

df_chamados_com_nivel = (
    df_chamados_src
    .join(df_atendentes_src, "id_atendente", "inner")
    .select(
        "id_chamado",
        "id_atendente",
        "nivel_atendimento",
        "tempo_atendimento_minutos",
        "tempo_espera_minutos"
    )
)

df_tempo_por_nivel = (
    df_chamados_com_nivel
    .groupBy("nivel_atendimento")
    .agg(
        F.round(F.avg("tempo_atendimento_minutos"), 2).alias("tempo_medio_atendimento"),
        F.round(F.avg("tempo_espera_minutos"), 2).alias("tempo_medio_espera"),
        F.count("id_chamado").alias("total_chamados")
    )
    .orderBy("nivel_atendimento")
)


df_tempo_por_nivel_final = (
    df_tempo_por_nivel
    .join(
        taxa_por_nivel.select(
            "nivel_atendimento",
            "taxa_resolucao_media"
        ),
        on="nivel_atendimento",
        how="left"
    )
    .orderBy("nivel_atendimento")
)

df_tempo_por_nivel_final_typed = (
    df_tempo_por_nivel_final
    .withColumn("nivel_atendimento", F.col("nivel_atendimento").cast(IntegerType()))
    .withColumn("tempo_medio_atendimento", F.col("tempo_medio_atendimento").cast(DecimalType(10,2)))
    .withColumn("tempo_medio_espera", F.col("tempo_medio_espera").cast(DecimalType(10,2)))
    .withColumn("taxa_resolucao_media", F.col("taxa_resolucao_media").cast(DecimalType(10,2)))
    .withColumn("total_chamados", F.col("total_chamados").cast(IntegerType()))
)


###Gravação na camada Gold
Salvar tempo_por_nivel

In [0]:
df_tempo_por_nivel_final_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_tempo_por_nivel)

gold_table_tempo_por_nivel = spark.table(tgt_table_tempo_por_nivel)
final_count_tempo_por_nivel = gold_table_tempo_por_nivel.count()

print(f"Salvo em: {tgt_table_tempo_por_nivel}")
print(f"Contagem final: {final_count_tempo_por_nivel:,}")

display(gold_table_tempo_por_nivel.limit(10))